<a href="https://colab.research.google.com/github/lionelsu/XPE_Python/blob/main/Async.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install nest_asyncio
import nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
nest_asyncio.apply()
import asyncio
from multiprocessing import Pool, cpu_count
import concurrent.futures
import sys
import time
from enum import Enum
import aiohttp
import requests

In [23]:
def get_ids_cervejarias():
    r = requests.get('https://api.openbrewerydb.org/breweries')
    if r.status_code == 200:
        return [r['id'] for r in r.json()]

In [24]:
def get_cervejaria(id_c):
    r = requests.get('https://api.openbrewerydb.org/breweries/{id_c}')
    if r.status_code == 200:
        print(r.json())

In [25]:
async def get_cervejaria_async(id_c, session):
    async with session.get(f'https://api.openbrewerydb.org/breweries/{id_c}') as response:
        print(await response.json())

In [26]:
async def processar_async(ids):
    coros = []
    async with aiohttp.ClientSession() as session:
        for id_c in ids:
            coros.append(get_cervejaria_async(id_c, session))

        await asyncio.gather(*coros)

In [27]:
async def get_cervejaria_async_manual(id_c):
    loop = asyncio.get_running_loop()
    return await loop.run_in_executor(Nome, get_cervejaria, id_c)

In [28]:
class TipoExecucao(Enum):
    UM_PROCESSO = 1
    VARIAS_THREADS = 2
    ASYNCIO_COM_LIB_HTTP = 3

In [34]:
if __name__ == '__main__':
    started = time.time()

    tipo_execucao = TipoExecucao.VARIAS_THREADS
    cores = cpu_count()

    ids_cervejarias = get_ids_cervejarias()

    if TipoExecucao.UM_PROCESSO == tipo_execucao:
        for id_cervejaria in ids_cervejarias:
            get_cervejaria(id_cervejaria)

    elif TipoExecucao.VARIAS_THREADS == tipo_execucao:
        with concurrent.futures.ThreadPoolExecutor(cores) as thp:
            thp.map(get_cervejaria, ids_cervejarias)
        
    elif TipoExecucao.ASYNCIO_COM_LIB_HTTP == tipo_execucao:
        event_loop = asyncio.get_event_loop()
        tasks = []

        event_loop.run_until_complete(processar_async(ids_cervejarias))
    elif TipoExecucao.ASYNCIO_MANUAL == tipo_execucao:
        event_loop = asyncio.get_event_loop()
        tasks = []

        for id_cervejaria in ids_cervejarias:
            tasks.append(get_cervejaria_async_manual(id_cervejaria))

        event_loop.run_until_complete(asyncio.wait(tasks))

    elapsed = time.time()
    print('Time taken :', elapsed - started)


Time taken : 0.8083250522613525
